In [2]:
import pandas as pd
import xgboost as xgb
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from xgboost.sklearn import XGBClassifier  
import numpy as np


player = pd.read_csv('./data/player.csv')
playerNoSal = pd.read_csv('./data/playerNoSal.csv')
playerScale = pd.read_csv('./data/playerscale.csv')
playerNoSalScale = pd.read_csv('./data/playerNoSalscale.csv')

/Users/patriciagoresen/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
player.head()
player = player.drop(['Unnamed: 0'], axis=1)
playerNoSal.head()
playerNoSal = playerNoSal.drop(['Unnamed: 0'], axis=1)
playerScale.head()
playerScale = playerScale.drop(['Unnamed: 0'], axis=1)
playerNoSalScale.head()
playerNoSalScale = playerNoSalScale.drop(['Unnamed: 0'], axis=1)

In [11]:
print(player.info())
print(playerNoSal.info())
print(playerScale.info())
print(playerNoSalScale.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19820 entries, 0 to 19819
Data columns (total 43 columns):
yearID                 19820 non-null int64
playerID               19820 non-null object
G_x                    19820 non-null int64
AB                     19820 non-null int64
R                      19820 non-null int64
H                      19820 non-null int64
2B                     19820 non-null int64
3B                     19820 non-null int64
HR                     19820 non-null int64
RBI                    19820 non-null float64
SB_x                   19820 non-null float64
CS_x                   19820 non-null float64
BB                     19820 non-null int64
SO                     19820 non-null float64
IBB                    19820 non-null float64
HBP                    19820 non-null float64
SH                     19820 non-null float64
SF                     19820 non-null float64
GIDP                   19820 non-null float64
stint                  19820 non-nul

In [4]:
#Seperate Predicter 
X = playerScale.drop(['Primary', 'playerID'], axis=1)
y = playerScale['Primary']
yhat = np.empty(y.shape, dtype=str)

In [5]:
for feature in X.columns: 
    if X[feature].dtype == 'object': 
        X[feature] = pd.Categorical(combined_set[feature]).codes 

In [11]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19820 entries, 0 to 19819
Data columns (total 38 columns):
G_x               19820 non-null float64
AB                19820 non-null float64
R                 19820 non-null float64
H                 19820 non-null float64
2B                19820 non-null float64
3B                19820 non-null float64
HR                19820 non-null float64
RBI               19820 non-null float64
SB_x              19820 non-null float64
CS_x              19820 non-null float64
BB                19820 non-null float64
SO                19820 non-null float64
IBB               19820 non-null float64
HBP               19820 non-null float64
SH                19820 non-null float64
SF                19820 non-null float64
GIDP              19820 non-null float64
stint             19820 non-null float64
G_y               19820 non-null float64
GS                19820 non-null float64
InnOuts           19820 non-null float64
PO                19820 non-nu

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
import xgboost as xgb
from sklearn.metrics import accuracy_score


# Simple XGBoost Model

In [63]:
xgbModel = XGBClassifier()
xgbModel.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [64]:
y_pred = xgbModel.predict(X_test)

In [65]:
accuracy = accuracy_score(y_test, y_pred)
print("The Accuracy of the XGBoost Model: %.2f%%" % (accuracy * 100.0))

The Accuracy of the XGBoost Model: 87.61%


In [15]:
# Now with cross validation 
DM_train = xgb.DMatrix(X_train.values, y_train.values) 
DM_test =  xgb.DMatrix(X_test,y_test)

gbm_param_grid = {
     'colsample_bytree': np.linspace(0.5, 0.9, 5),
     'n_estimators':[100, 200]
}

gbm = xgb.XGBRegressor()
gbm_grid = GridSearchCV(estimator = gbm, param_grid = gbm_param_grid, scoring = 'neg_mean_squared_error', cv = 10, verbose = 1)
gbm_grid.fit(X_train, y_train)

print("Best parameters found: ",grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))
print("Best score: %s" % (grid_mse.best_score_))



pred = gbm_grid.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print("The Accuracy of the XGBoost Model With Cross-Validation: %.2f%%" % (accuracy * 100.0))

ValueError: could not convert string to float: '2B'

In [7]:
from sklearn.cross_validation import StratifiedShuffleSplit
cv = StratifiedShuffleSplit(y,n_iter = 10, test_size=0.2, train_size = 0.8)
gbm = xgb.XGBRegressor()
gbm_param_grid = {
     'colsample_bytree': np.linspace(0.5, 0.9, 5),
     'n_estimators':[100, 200]
}
gbm_grid = GridSearchCV(estimator = gbm, param_grid = gbm_param_grid, scoring = 'neg_mean_squared_error', cv = 10, verbose = 1)

for traindx, testdx in cv:
    X_train = X[traindx]
    X_test= X[testdx]
    y_train = y[traindx]
    y_test = t[testdx]
    gbm_grid.fit(X_train, y_train)

KeyError: '[17480  3047  3623 ..., 11651  2789  3131] not in index'